In [2]:
from datetime import datetime, timedelta
import pandas as pd
from openpyxl import load_workbook
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

In [3]:
pio.templates.default = "gridon"

In [4]:
def extract_date(date_str):
    return date_str.split('T')[0]


# Carica il tuo file Excel e prepara i dati come hai fatto nel tuo codice
# originale
file_excel = "data/Books.xlsx"
df = pd.read_excel(file_excel, sheet_name="Shelf")
df = df.drop('Cover', axis=1)

In [5]:
df.head(3)

,Evaluation,Status,Title,Authors,Start,End,Type,Category,Sub-Category,Series,Language
0,8.0,Read,C++ Crash Course,"Lospinoso,Josh",NaT,NaT,ebook,Coding,C++,NaN,en
1,8.0,Read,Beautiful C++,"Davidson,J.;Gregory,Kate",NaT,NaT,ebook,Coding,C++,NaN,en
2,7.5,Read,Calcoli morali,"Mérö,László",NaT,NaT,paper,Psychology,Game theory,NaN,it


In [6]:
colors = ['green', 'yellow', 'blue', 'red']
status_counts = df['Status'].value_counts()

fig = go.Figure(data=[go.Pie(labels=status_counts.index,
                             values=status_counts.values)])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=30,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.show()

In [7]:
df.columns

Index(['Evaluation', 'Status', 'Title', 'Authors', 'Start', 'End', 'Type',
       'Category', 'Sub-Category', 'Series', 'Language'],
      dtype='object')

In [11]:
# Copia il DataFrame originale per evitare modifiche indesiderate
df_copy = df.copy()

# Sostituisci le date End mancanti con la data di oggi
today = datetime.today().date()
df_copy['End'].fillna(today, inplace=True)

# Crea il Gantt chart con Plotly Express
fig = px.timeline(df_copy[df_copy['Start'].notna()], x_start='Start', x_end='End', y='Title', color='Category', hover_data=['Authors', 'Type', 'Sub-Category', 'Language', 'Status'])

fig.update_yaxes(categoryorder="total ascending")

fig.update_layout(
    xaxis_title="",
    yaxis_title="",
    title="",
    showlegend=True,
    margin=dict(l=250)
)

fig.update_traces(marker=dict(line=dict(width=2)))  # Modifica la larghezza delle barre

fig.show()

In [8]:
df.columns

Index(['Evaluation', 'Status', 'Title', 'Authors', 'Start', 'End', 'Type',
       'Category', 'Sub-Category', 'Series', 'Language'],
      dtype='object')

In [15]:
color_discrete_map = {category: px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)]
                      for i, category in enumerate(df['Category'].unique())}


fig = px.treemap(df, 
                 path=['Category', 'Sub-Category', 'Title'],  # Utilizza 'Category' come macro-categoria e 'Sub-Category' come sotto-categoria
                 values='Evaluation',  # Valori da rappresentare nel treemap
                 color='Category',  # Colore basato sulla colonna 'Status'
                 color_discrete_map=color_discrete_map,
                 maxdepth=2,
                 title='')  # Titolo del grafico

fig.update_traces(marker=dict(cornerradius=5))

# Mostra il treemap
fig.show()

In [10]:
df.head(3)

,Evaluation,Status,Title,Authors,Start,End,Type,Category,Sub-Category,Series,Language
0,8.0,Read,C++ Crash Course,"Lospinoso,Josh",NaT,NaT,ebook,Coding,C++,NaN,en
1,8.0,Read,Beautiful C++,"Davidson,J.;Gregory,Kate",NaT,NaT,ebook,Coding,C++,NaN,en
2,7.5,Read,Calcoli morali,"Mérö,László",NaT,NaT,paper,Psychology,Game theory,NaN,it


In [14]:
read = df[df['Status'] == 'Read']
   
# Crea il treemap utilizzando le medie calcolate come colore
fig = px.treemap(read, 
                 path=['Category', 'Sub-Category','Title'],  # Utilizza solo 'Category' e 'Sub-Category' come livelli
                 values='Evaluation',  # Valori da rappresentare nel treemap
                 color='Evaluation',  # Colore basato sulla media di 'Evaluation'
                 color_continuous_scale=[[0, 'red'], [0.5, 'white'], [1, 'green']],
                 maxdepth=2,
                 title=''
                 ) 

fig.update_traces(marker=dict(cornerradius=5))
fig.update_traces(marker=dict(line=dict(width=0)))
fig.show()